## equity price in sector

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline


# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

from IPython.display import display
# Setting display options to show all rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

import os
import sys
import seaborn as sns
# 
# 
from quant_free.dataset.equity_load import *
from quant_free.utils.us_equity_utils import *

from quant_free.factor.xq_finance import *
from quant_free.dataset.equity_load import *


In [ ]:

# sector_name = "IT_Services"
# sector_name = "Entertainment"
# sector_name = "Semiconductors_&_Semiconductor_Equipment"
# sector_name = "Automobiles"
# sector_name = "Internet_Retail"
sector_name = 'Trading_Companies_&_Distributors'

symbols = symbol_load(market = 'us')

symbols = us_dir1_load_csv(market = 'us', dir0 = 'symbol', dir1 = 'fh', filename= sector_name +'.csv')['symbol'].values

print("symbol list len ", len(symbols))

factors = ['roe', 'roa', 'gross_profit_revenue', 'profit_revenue', 'revenue_increase_q2q_rate', 'cash_increase_q2q_rate', 'profit_increase_q2q_rate']
start_date = '2023-06-01'
end_date = '2024-06-01'
us_analysis_finance = xq_finance(symbols, factors,  start_date, end_date)
result = us_analysis_finance.finance_factors_rank(factors)

us_equity_research_folder("finance", sector_name + start_date + "_" + end_date + ".csv", result[2])

# us_equity_research_folder("finance", 'finance_factors_rank.csv', result)
# print(result)
display(result[0])
display(result[1])
display(result[2])

In [ ]:
from quant_free.utils.us_equity_utils import *

# symbols = ["AMZN", "AAPL", "NFLX", "GOOG"]
symbols = result[0].index
# symbols = symbol_load(market = 'us').head(2000)
# data = us_equity_daily_data_load(symbols = symbols, option = 'close')

start_date = '2024-01-29'
end_date = '2024-07-08'

data = us_equity_daily_data_load(symbols = symbols, start_date = start_date, end_date = end_date, column_option = 'close')
data = pd.DataFrame(data)
data = data.sub(data.iloc[0, :], axis=1).div(data.iloc[0, :], axis=1) * 100

sorted_columns = data.iloc[-1].sort_values(ascending=False).index
data = data[sorted_columns]
# data = data.loc[:, data.iloc[-1] > 30]

file = sector_name + start_date + "_" + end_date + ".csv"
folder = us_equity_research_folder(sub_folder = "price", file_name = file, data = data)

print(data.tail(1).transpose())

# Convert the DataFrame to long format
df_long = data.reset_index().melt(id_vars='index', var_name='variable', value_name='value')

# Create a line plot using Seaborn's relplot
g = sns.relplot(
    data=df_long, 
    x='index', 
    y='value', 
    hue='variable', 
    kind='line',
    style='variable',  # Different line styles for each variable
    height=10,    # Height of each facet
    aspect=1.5  # Aspect ratio of each facet
)


# Rotate x-axis labels and set only a subset of ticks
for ax in g.axes.flat:
    x = data.index[1:-1:20]
    ax.set_xticks(x)  # Set the desired tick positions
    ax.set_xticklabels(x)  # Set the desired tick labels
    for label in ax.get_xticklabels():
        label.set_rotation(90)